In [5]:
# Auto-reload changes to the backend code
%load_ext autoreload
%autoreload 2

import sys
import os
import asyncio

# Add the backend directory to Python path
current_dir = os.getcwd()
backend_dir = os.path.join(current_dir, '..', '..')
sys.path.insert(0, os.path.abspath(backend_dir))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
filepath = r"C:\Users\danse\Projects\FleetAI V1\Training Data\Contracts\Ben Gurion\fuel_contract_expanded.pdf"

In [15]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from app.ai.rag.rag_pipeline import rag_pipeline


# 1. Run pipeline to get docs + embeddings
docs = await rag_pipeline(filepath)

# 2. Convert them into LangChain Document objects
from langchain_core.documents import Document
documents = [
    Document(page_content=doc["text"], metadata=doc["metadata"])
    for doc in docs
]

# 3. Build FAISS vector store in memory
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = FAISS.from_documents(documents, embeddings)

# 4. Make a retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# 5. Create a RetrievalQA chain
llm = ChatOpenAI(model="gpt-4o-mini")  # Or whichever model you want
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
)

# 6. Ask a question
query = "What are the payment terms in this contract?"
result = qa_chain.invoke({"query": query})

print("Answer:", result["result"])
print("Sources:", result["source_documents"])


ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).